# This is for plotting wind profile under neutral condition 
2022-3-30

In [1]:
# import packages
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import os.path
import pickle
import time
from matplotlib.pyplot import figure
import scipy.io as sio
from datetime import date, timedelta
from mpmath import *
# directories
IN_DIRECTORY = "/projectnb/urbanclimate/yueqin/idaho_2020/basic_processed_data/"
IN_DIRECTORY2 = "/projectnb/urbanclimate/yueqin/idaho_2020/processeddata/"
msk_DIR = "/projectnb/urbanclimate/yueqin/idaho_2020/mask/"
OUT_DIRECTORY = "/projectnb/urbanclimate/yueqin/idaho_2020/fig_neutral/"
ins_var=['u_ins'] # 36000 instantaneous variables per hour 
tur_var=['u_tur'] # 36000 instantaneous variables per hour 
avg_var=['u_avg','u_star'] # 1 averaged variable per hour
flg_var=['diag_csat_ins','sig_irga_ins']
msk = ['mask_INL_all_1d']

In [39]:
%%time
Dir = "/projectnb/urbanclimate/yueqin/idaho_2020/compressed_data/"
b_file = open('/projectnb/urbanclimate/yueqin/idaho_2020/basic_processed_data/w_ins_20200920.pkl', "rb")
w = pickle.load(b_file)    
b_file.close()
b_file = open('/projectnb/urbanclimate/yueqin/idaho_2020/basic_processed_data/u_ins_20200920.pkl', "rb")
u = pickle.load(b_file)    
b_file.close()
b_file = open('/projectnb/urbanclimate/yueqin/idaho_2020/basic_processed_data/v_ins_20200920.pkl', "rb")
v = pickle.load(b_file)    
b_file.close()
np.savez(Dir+'ins_20200920.npz',u=u,v=v,w=w)

CPU times: user 341 ms, sys: 268 ms, total: 609 ms
Wall time: 3.93 s


In [45]:
%%time
b_file = open('/projectnb/urbanclimate/yueqin/idaho_2020/basic_processed_data/w_ins_20200920.pkl', "rb")
a = pickle.load(b_file)    
b_file.close()
w = a

CPU times: user 1.94 ms, sys: 29.4 ms, total: 31.3 ms
Wall time: 30 ms


In [49]:
%%time
a = np.load('/projectnb/urbanclimate/yueqin/idaho_2020/compressed_data/w_ins_20200920.npz')
w = a['arr_0']

CPU times: user 96.2 ms, sys: 27.2 ms, total: 123 ms
Wall time: 125 ms


In [29]:
!ls -lh /projectnb/urbanclimate/yueqin/idaho_2020/basic_processed_data/w_ins_20200920.pkl
!ls -lh /projectnb/urbanclimate/yueqin/idaho_2020/compressed_data/w_ins_20200920.npz

-rw-r--r-- 1 yueqin urbanclimate 80M Sep 23  2021 /projectnb/urbanclimate/yueqin/idaho_2020/basic_processed_data/w_ins_20200920.pkl
-rw-r--r-- 1 yueqin urbanclimate 80M Mar 31 11:55 /projectnb/urbanclimate/yueqin/idaho_2020/compressed_data/w_ins_20200920.npz


# define constants and labels

In [50]:
sonum    =12                       # number of sonic
z        = np.array([1.2,2,3.5,6,8.2,12.8,15.8,23,30.3,40.2,50.6,60.5])                # height of sonic above ground, m
frequency=10                   # sampling frequency, Hz
time_avg =3600                  # average time, s
rpat = time_avg*frequency           # number of lines for a loop

# physical constants (or values that are assumed constant)
Rw  = 461.5     # ideal gas constant for water vapor, J/kg*K
Rd  = 287.05    # ideal gas constant for dry air, J/kg*K
Lv  = 1000*2257 # latent heat of vaporization (water), J/kg
Cp  = 1005      # approximate constant pressure specific heat of air, J/kg*K
kappa   = 0.4       # Von Karman constant
g   = 9.81      # acceleration of gravity, m/s^2
list_col = ['C1','C2','C3','C4','B1','B2','B3','B4','A1','A2','A3','A4']

# selected levels
list_ind = range(5,11) # from level 6 to level 11

# define functions

In [51]:
def date_list(sdate,edate):
    """method used for creating date list"""
    delta = edate - sdate       # as timedelta
    day = [sdate+timedelta(days=x) for x in range(delta.days+1)]
    return day

# set up time period and initialize variables

In [71]:
Sdate = date(2020,9,25)
Edate = date(2021,4,23)
ds = date_list(Sdate,Edate)
# u_ins_df = np.zeros((24*rpat,sonum,len(ds)))+np.nan
# v_ins_df = np.zeros((24*rpat,sonum,len(ds)))+np.nan
# w_ins_df = np.zeros((24*rpat,sonum,len(ds)))+np.nan
# u_tur_df = np.zeros((24*rpat,sonum,len(ds)))+np.nan
# v_tur_df = np.zeros((24*rpat,sonum,len(ds)))+np.nan
# w_tur_df = np.zeros((24*rpat,sonum,len(ds)))+np.nan
# ts_df = np.zeros((24*rpat,len(ds)))+np.nan

# set up controls

In [53]:
do_processing = True
do_saving = False

# Load instantaneous data and process data

In [72]:
%%time
t_id = 0
i_hr_total = 0
for day in ds:
    strday = str(day.strftime("%Y%m%d"))
    fp_stats = IN_DIRECTORY + 'u_ins_' + strday +'.pkl'
    if (not os.path.isfile(fp_stats)):
        print(day.strftime("%Y%m%d")+' do not exist')
        continue
    print('start loading:'+ strday)
    # load neutral mask
    b_file = open(msk_DIR + 'mask_neutral_' + strday +'.pkl', "rb")
    mask_neutral = pickle.load(b_file)    
    b_file.close()
    if not any(mask_neutral):
        continue
    # load masks
    for var in msk:
        a_file = open(msk_DIR + var +'_' + strday +'.pkl', "rb")
        globals()[var] = pickle.load(a_file)    
        a_file.close()
    nan_msk=np.isnan(mask_INL_all_1d)
    if all(nan_msk):
        continue
    # load avg variables
    for var in avg_var: 
        a_file = open(IN_DIRECTORY + var +'_' + strday +'.pkl', "rb")
        globals()[var] = pickle.load(a_file)    
        a_file.close() 
    # compute log wind profile
    if do_processing:
        print('start processing:'+ strday)
        slope_m1_sel = np.zeros((len(mask_neutral),1))+np.nan
        intecept_m1_sel = np.zeros((len(mask_neutral),1))+np.nan
        zo_m1_sel = np.zeros((len(mask_neutral),1))+np.nan
        u_star_m1_sel = np.zeros((len(mask_neutral),1))+np.nan
        u_stat_dev = np.zeros((len(mask_neutral),sonum))+np.nan
        u_star_m1_sel_dev = np.zeros((len(mask_neutral),sonum))+np.nan
        u_star_m1_sel_dev_mean = np.zeros((len(mask_neutral),1))+np.nan
        for i in range(len(mask_neutral)):
            if mask_neutral[i] and (~nan_msk[i]):
                print('neutral case - Hour ' + str(i+1))
                # compute variations of u_star across all levels, the mean value is calculated by selected levels
                u_stat_dev[i,:] = (u_star[i,:]-np.nanmean(u_star[i,list_ind]))/np.nanmean(u_star[i,list_ind])
                # ---------------------- method 1, fit with kappa = 0.4 and using selected data
                p1,p2 = np.polyfit(np.log(z[list_ind]), u_avg[i,list_ind], deg = 1)
                slope_m1_sel[i]    = p1
                intecept_m1_sel[i] = p2
                zo_m1_sel[i]   = exp(-p2/p1) # momentum roughness length
                u_star_m1_sel[i] = kappa*slope_m1_sel[i]
                # compute variations of u_star across all levels, the mean value is calculated by method 1
                u_star_m1_sel_dev[i,:] = (u_star[i,:]-u_star_m1_sel[i])/u_star_m1_sel[i]
                u_star_m1_sel_dev_mean[i] = np.nanmean(abs(u_star_m1_sel_dev[i,list_ind]))
                
#                 x_vector = log(z[list_ind])
#                 y_vector = u_avg[i,list_ind]
#                 R2_method1_sel (kk,1) = calculateR2(y_vector,x_vector);
#                 R2_method1_sel_test (kk,1) = calculateR2(y_vector./u_star_method1_sel(kk,1),x_vector); %this will not the results
#                 RMSE_method1_sel(kk,1) =  sqrt(sum((y_vector-(slope_method1_sel(kk,1)*x_vector+intecept_method1_sel(kk,1))).^2)); 
                
                # ---------------------- method2, fit with ec measured ustar and using selected data
                # This method will give nearly the same zo as method 1 if using
                # the same selected data, the only difference is the kappa value and the u_star
                
        


#         cdatetime = cyr +'-' + cmonth +'-'+cdys
#         st_datetime = pd.Timestamp(int(cyr),int(cmonth),int(cdys),0)
#         ed_datetime = pd.Timestamp(int(cyr),int(cmonth),int(cdys),23)
#         DateTime = pd.date_range(start=st_datetime, end=ed_datetime, periods=24)
#         ts = np.repeat(DateTime,rpat)
#         ts_df[:,t_id]=ts
#         u_ins_df[:,:,t_id]=u_ins
#         v_ins_df[:,:,t_id]=v_ins
#         w_ins_df[:,:,t_id]=w_ins

    print('finish processing:'+ strday)
    print('------------------------')
    t_id += 1

start loading:20200925
start processing:20200925
neutral case - Hour 16
neutral case - Hour 17
neutral case - Hour 19
finish processing:20200925
------------------------
start loading:20200926
start loading:20200927
start loading:20200928
start loading:20200929
start loading:20200930
start loading:20201001
start loading:20201002
start loading:20201003
start loading:20201004
start loading:20201005
start loading:20201006
start loading:20201007
start loading:20201008
start processing:20201008
finish processing:20201008
------------------------
start loading:20201009
start loading:20201010
start processing:20201010
neutral case - Hour 16
neutral case - Hour 21
finish processing:20201010
------------------------
start loading:20201011
start processing:20201011
finish processing:20201011
------------------------
start loading:20201012
start processing:20201012
neutral case - Hour 17
neutral case - Hour 18
finish processing:20201012
------------------------
start loading:20201013
start proces

FileNotFoundError: [Errno 2] No such file or directory: '/projectnb/urbanclimate/yueqin/idaho_2020/mask/mask_neutral_20201023.pkl'

In [34]:
list_ind = range(5,11)
z        = np.array([1.2,2,3.5,6,8.2,12.8,15.8,23,30.3,40.2,50.6,60.5])
print(z[list_ind])

[12.8 15.8 23.  30.3 40.2 50.6]
